# Get list of public protected datasets

In [ ]:
import hubmapbags
from datetime import datetime
from pathlib import Path
import pandas as pd
from tqdm import tqdm
from warnings import warn as warning

token = 'token'
instance = 'prod' #default instance is test

In [ ]:
# get assay types
assay_names = hubmapbags.get_assay_types()

In [ ]:
report = pd.DataFrame()
for assay_name in assay_names:
    print(assay_name)
    datasets = pd.DataFrame(hubmapbags.get_hubmap_ids( assay_name=assay_name, token=token ))

    if datasets.empty:
        continue
    
    #clean up
    datasets = datasets[(datasets['data_type'] != 'image_pyramid')]
    datasets = datasets[(datasets['status'] == 'Published')]
    
    datasets['is_protected'] = None
    for index, datum in tqdm(datasets.iterrows()):
        datasets.loc[index,'is_protected'] = hubmapbags.apis.is_protected( datum['hubmap_id'], instance='prod', token=token )
        
        if datasets.loc[index,'is_protected']:
            datasets.loc[index,'number_of_uuids'] = hubmapbags.uuids.get_number_of_uuids( datum['hubmap_id'], instance='prod', token=token )
    
    if report.empty:
        report = datasets
    else:
        report = pd.concat( [report, datasets] )
        
report = report[report['is_protected'] == True]

In [ ]:
from datetime import datetime
now = datetime.now() 

if not Path('protected-datasets-report').exists():
    Path('protected-datasets-report').mkdir()
report.to_csv('protected-datasets-report/' + str(now.strftime('%Y%m%d')) + '.tsv', sep='\t', index=False )